# Example GAN training
Tutorial which shows how to start a training

first run 'cd ../..' to be in the root directory so that you can import custom packages via notebook

In [1]:
cd ../..

/Users/hao/workspace/hpi_de/4th_Semester/Applied Machine Learning/sensor-data-gans


## import

In [2]:
from gans.gan_trainer import GanTrainer

from datetime import datetime
from numpy.random import randn

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv1D, Flatten, Reshape, Conv1DTranspose
from tensorflow.keras.optimizers import Adam

## Initialize GanTrainer

In [3]:
out_dir = 'test'
gan_trainer = GanTrainer(out_dir=out_dir)

## Load and transform Data

In [4]:
# data parameters
train_path = './datasets/mydata/train_df.h5'
val_path = './datasets/mydata/val_df.h5'
test_path = './datasets/mydata/test_df.h5'

# what action to generate
act_id = 0

# preprocessing parameters
window_size = 5*50
step_size = int(window_size/2)
method ='sliding'
col_names = ['userAcceleration.x', 'userAcceleration.y', 'userAcceleration.z', 'userAcceleration.c']
input_cols_train=['userAcceleration.x', 'userAcceleration.y', 'userAcceleration.z']
input_cols_eval=['userAcceleration.c']

In [5]:
gan_trainer.init_data(train_path=train_path,
                      val_path=val_path,
                      test_path=test_path,
                      act_id=act_id,
                      window_size=window_size,
                      step_size=step_size,
                      method=method,
                      col_names=col_names,
                      input_cols_train=input_cols_train,
                      input_cols_eval=input_cols_eval
                     )

Load Data...
Transform Data...
Calculate origin performance...
Done!


## create GAN

### create discriminator

In [6]:
input_shape=(250, 3)

discriminator = Sequential()
discriminator.add(Conv1D(8, kernel_size=9, activation='relu', input_shape=input_shape))
discriminator.add(Flatten())
discriminator.add(Dense(1, activation='sigmoid'))

discriminator.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [7]:
discriminator.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 242, 8)            224       
_________________________________________________________________
flatten (Flatten)            (None, 1936)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1937      
Total params: 2,161
Trainable params: 2,161
Non-trainable params: 0
_________________________________________________________________


### create generator

In [8]:
latent_dim=64

generator = Sequential()
generator.add(Dense(125 * 3, kernel_initializer='he_uniform', input_shape=(latent_dim,)))
generator.add(Reshape((125, 3)))
generator.add(Conv1D(3, kernel_size=5, kernel_initializer='he_uniform', padding='same'))
generator.add(Conv1DTranspose(3, kernel_size=4, strides=2, padding='same'))
generator.add(Conv1D(3, kernel_size=5, padding='same', kernel_initializer='he_uniform', activation='linear'))

In [9]:
generator.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 375)               24375     
_________________________________________________________________
reshape (Reshape)            (None, 125, 3)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 125, 3)            48        
_________________________________________________________________
conv1d_transpose (Conv1DTran (None, 250, 3)            39        
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 250, 3)            48        
Total params: 24,510
Trainable params: 24,510
Non-trainable params: 0
_________________________________________________________________


### combine generator and discriminator to GAN

In [10]:
gan_trainer.create_gan(generator=generator, discriminator=discriminator)

## Training

In [11]:
# training parameters
steps = 100
batch_size = 32
eval_step = 10
random = False
label_smoothing = False

In [12]:
gan_trainer.train_gan(steps=steps, 
                    batch_size=batch_size,
                    eval_step=eval_step,
                    random=random, 
                    label_smoothing=label_smoothing)

discriminator loss: [1.0480483770370483, 0.484375]
adversarial loss: 1.090978980064392


discriminator loss: [0.360035240650177, 0.5]
adversarial loss: 6.771139144897461


discriminator loss: [0.3945339620113373, 0.53125]
adversarial loss: 7.51900577545166


discriminator loss: [0.3444904088973999, 0.875]
adversarial loss: 6.824872016906738


discriminator loss: [0.346714049577713, 0.875]
adversarial loss: 5.413257598876953


discriminator loss: [0.3426848351955414, 1.0]
adversarial loss: 4.9119462966918945


Train f1-score for act_id  0 improved from  0.0 to  0.9333333333333333
Validation f1-score for act_id  0 improved from  0.0 to  0.8523290386521308
discriminator loss: [0.3472014367580414, 1.0]
adversarial loss: 4.131976127624512


Train f1-score for act_id  0 improved from  0.9333333333333333 to  0.9596774193548387
Validation f1-score for act_id  0 improved from  0.8523290386521308 to  0.9851088201603665
discriminator loss: [0.34426814317703247, 1.0]
adversarial loss: 3.9074804782